In [1]:
import os, sys
import numpy as np
from tools import *
from image_processing import *
from optparse import OptionParser

from model import *

from keras import backend


Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: Quadro M4000 (0000:00:05.0)


In [2]:
limit = 11500
backend.set_image_dim_ordering('th')
IMG_SIZE = 224
classes = get_classes()

# Definition des fonctions

In [51]:
def load_pictures(path, filename, limit, forceProcess = False):
    if os.path.exists(filename) and not forceProcess:
        preprocessed_imgs = restore(filename)
        print ("Done.")
    else:
        print ("Preprocessing....")
        preprocessed_imgs = preprocess_imgs(path, IMG_SIZE, limit = limit)
        store(preprocessed_imgs, filename)
    return preprocessed_imgs
        

def load_dataset(path, filename, limit, forceProcess = False):
    cats_file = filename + 'cats.pckl'
    dogs_file = filename + 'dogs.pckl'
    print ("Loading cats images:")
    preprocessed_cats = load_pictures(path, cats_file, limit, forceProcess)
    print ("Loading dogs images:")
    preprocessed_dogs = load_pictures(path, dogs_file, limit, forceProcess)
    return preprocessed_valid_cats, preprocessed_valid_dogs
    

def load_validation_pictures(limit_valid, forceProcess = False):
    return load_dataset('valid', 'preprocessed_validation_', limit_valid, forceProcess = False)


def load_train_pictures(limit, forceProcess = False):
    return load_dataset('train', 'preprocessed_', limit_valid, forceProcess = False)

def construct_input_matrix(preprocessed_cats, preprocessed_dogs):   
    X = np.concatenate((preprocessed_cats, preprocessed_dogs), axis = 0)
    X = np.swapaxes(X, 1, 3)
    return X

def create_expected_output(limit):
    """labelling the data: cats = [1 0] and dogs = [0 1]"""
    zeros = np.zeros((limit, 1))
    ones  = np.ones((limit, 1))
    Ycats = np.array((ones, zeros))
    Ydogs = np.array((zeros, ones))

    Y = np.concatenate((Ycats, Ydogs), axis=1)
    Y = np.swapaxes(Y, 0, 1).squeeze()
    return Y



def predict_cat_dog_at_index(model, input_array, index, limit = limit):
    print ("Predict a cat: ")
    cat_prediction = model.predict(np.array([input_array[index,:,:]])).argmax(axis = 1)[0]
    print ( cat_prediction, classes[cat_prediction] )

    print ("Predict a dog: ")
    dog_prediction = model.predict(np.array([input_array[limit + index,:,:]])).argmax(axis = 1)[0]
    print (dog_prediction, classes[dog_prediction])
    
    
def convolutional_output(X, convolutional_model, forceProcess = False, filename = 'conv_output.pckl'):
    if os.path.exists('conv_output.pckl') and not forceProcess:
        conv_output = restore('conv_output.pckl')
    else:
        conv_output = convolutional_model.predict(X)
        store(conv_output, 'conv_output.pckl')
    return conv_output

# Load data

In [52]:
preprocessed_cats, preprocessed_dogs = load_train_pictures(limit)

Loading cats images:
Done.
Loading dogs images:
Done.


In [5]:
Y = create_expected_output(limit)
X = construct_input_matrix( preprocessed_cats, preprocessed_dogs )
print X.shape

(23000, 3, 224, 224)


# Construct and test Vgg model

In [45]:
model = create_model(IMG_SIZE)
for layer in model.layers: layer.trainable=False

In [46]:
predict_cat_dog_at_index(model, X, 4789)

Predict a cat: 
(281, u'tabby')
Predict a dog: 
(243, u'bull_mastiff')


# Sepate convolutional layers and dense layers


In [25]:
print ("Total number of layers : {}, only the 6 last layers are dense layers".format(len(model.layers)))

def no_process(x):
    # return x[:, ::-1]
    return x

def separate_models(model, nb_dense = 6):
    nb_layers = len(model.layers)
    dense_model = Sequential()
    dense_model.add(Lambda(no_process, input_shape=(512, 7, 7), output_shape=(512, 7, 7) ))
    for layer in model.layers[-6:]:
        if "dropout" not in layer.name:
            dense_model.add(layer)
    convolutional_model = Sequential(model.layers[:nb_layers - 6])
    print ("Total number of convolutional layers : {}".format(len(convolutional_model.layers)))
    print ("Total number of dense layers : {}".format(len(dense_model.layers)))
    return dense_model, convolutional_model

dense_model, convolutional_model = separate_models(model)

Total number of layers : 38, only the 6 last layers are dense layers
Total number of convolutional layers : 32
Total number of dense layers : 5


In [26]:
#convolutional_model.summary()

In [27]:
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              4097000   
Total params: 123,642,856
Trainable params: 0
Non-trainable params: 123,642,856
_________________________________________________________________


In [28]:
conv_output = convolutional_output(X, convolutional_model)

In [29]:
# check that the output still makes sense
predict_cat_dog_at_index(dense_model, conv_output, 4789)

Predict a cat: 
(281, u'tabby')
Predict a dog: 
(243, u'bull_mastiff')


In [196]:
im = Image.fromarray(np.swapaxes(X[12,:,:], 0 , 2) )
# im = Image.open("train/cats/cat.50.jpg")
im.show()

# Create validation set

In [53]:
limit_valid = 1000

preprocessed_valid_cats, preprocessed_valid_dogs = load_validation_pictures(limit_valid)
    

Loading cats images:
Done.
Loading dogs images:
Done.


In [31]:
dense_model.add(Dense(2, activation='softmax', input_shape=(1000,)))
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              4097000   
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2002      
Total params: 123,644,858
Trainable params: 2,002
Non-trainable params: 123,642,856
__________________________________________________________

In [32]:
Y_valid = create_expected_output(limit_valid)
X_valid = construct_input_matrix(preprocessed_valid_cats, preprocessed_valid_dogs)

Xtrain = X[0:2000,:,:,:]
Ytrain = Y[0:2000,:]

In [34]:
conv_valid_output = convolutional_output(X, convolutional_model, filename = 'conv_valid_output.pckl')


sample_valid_output = conv_valid_output[0:2000,:,:,:]

In [35]:
def generate_input_output(X, Y):
    def generate_arrays():
        for i in range(len(X)):
            yield (X[50*i:50*i+50,:,:,:],Y[50*i:50*i+50,:])
    return generate_arrays

In [104]:
dense_model.compile(
        # loss='binary_crossentropy',
       #optimizer=SGD(lr=10.0),
          #optimizer=RMSprop(),
       optimizer=Adam(lr=0.0001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])

dense_model.layers[-2].trainable = True

if os.path.exists('dense_model.h5'):
    dense_model.load_weights('dense_model.h5')
    
dense_model.fit(conv_output ,Y)

dense_model.save_weights("dense_model.h5")

Epoch 1/10
23000/23000 [==============================] - 8s - loss: 0.2225 - acc: 0.9672     
Epoch 2/10
23000/23000 [==============================] - 8s - loss: 0.2035 - acc: 0.9700     
Epoch 3/10
23000/23000 [==============================] - 8s - loss: 0.1872 - acc: 0.9725     
Epoch 4/10
23000/23000 [==============================] - 8s - loss: 0.1729 - acc: 0.9741     
Epoch 5/10
23000/23000 [==============================] - 8s - loss: 0.1602 - acc: 0.9756     
Epoch 6/10
23000/23000 [==============================] - 8s - loss: 0.1480 - acc: 0.9773     
Epoch 7/10
23000/23000 [==============================] - 8s - loss: 0.1383 - acc: 0.9784     
Epoch 8/10
23000/23000 [==============================] - 8s - loss: 0.1290 - acc: 0.9792     
Epoch 9/10
23000/23000 [==============================] - 8s - loss: 0.1201 - acc: 0.9804     
Epoch 10/10
23000/23000 [==============================] - 8s - loss: 0.1124 - acc: 0.9811     


In [ ]:
dense_model.compile(
        # loss='binary_crossentropy',
       #optimizer=SGD(lr=10.0),
          #optimizer=RMSprop(),
       optimizer=Adam(lr=0.0001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])


if os.path.exists('dense_model.h5'):
    model.load_weights('dense_model.h5')
    
dense_model.fit_generator(generate_input_output(convolutional_output, Y), 
                          steps_per_epoch=40, epochs=1, validation_steps = 40, 
                          validation_data = (sample_valid_output, Y_valid) )
dense_model.save_weights("dense_model.h5")

# Make predictions on the test set

In [98]:
test_set_size= 12500
preprocessed_test_set = load_pictures('test', 'preprocessed_test_set.pckl', test_set_size, forceProcess = False)

Done.


In [99]:
X_test = np.swapaxes(preprocessed_test_set, 1, 3)
print X_test.shape

(12500, 3, 224, 224)


In [100]:
test_conv_output = convolutional_output(X_test, convolutional_model, forceProcess = False, filename = 'test_conv_output.pckl')
print test_conv_output.shape

(12500, 512, 7, 7)


In [85]:
dense_model.predict(convolutional_model.predict(np.array([X_test[12]])))
dense_model.predict(np.array([test_conv_output[12]]))

array([[ 0.83419549,  0.16580449]], dtype=float32)

In [73]:
print dense_model.predict(convolutional_model.predict(np.array([X[5]])))
print dense_model.predict(np.array([conv_output[5]]))

print dense_model.predict(convolutional_model.predict(np.array([X_valid[50]])))
print dense_model.predict(np.array([conv_valid_output[50]]))

[[ 0.83685905  0.16314095]]
[[ 0.83685905  0.16314095]]
[[ 0.8367421   0.16325796]]
[[ 0.83685935  0.16314064]]


In [105]:
test_set_predictions = dense_model.predict(test_conv_output)
print test_set_predictions.shape

(12500, 2)


In [106]:
submission_string = "id,label\n"
for i,prediction in enumerate(test_set_predictions):
    submission_string = submission_string + str(i+1) + "," + str(prediction[1]) + "\n"

    
submission_file = open("Submission.csv", "w")
submission_file.write(submission_string)
submission_file.close()